In [ ]:
from flask import Flask, render_template, request
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import Keys
from selenium.webdriver.common.by import By
import os
import pandas as pd

In [ ]:
driver = webdriver.Chrome(keep_alive=True)
driver.set_window_size(1100, 800)
print("Clicker starts")
username = "info@waddystore.com"
password = "Waddy1208"
driver.get('https://admin.shoplineapp.com/admin/waddystore/')
driver.find_element(By.ID, "staff_email").send_keys(username)
driver.find_element(By.ID, "staff_password").send_keys(password)
driver.find_element(By.ID, "reg-submit-button").click()
print("Login sucessful")

In [ ]:
sku_id = "63649e44c6169500145493af"
print("Now browsing to SKU: " + sku_id)
driver.get("https://admin.shoplineapp.com/admin/waddystore/products/"+sku_id+"/edit")


driver.find_element(By.XPATH,'//*[@id="product_form"]/div[1]/div[3]/ul/li[4]/a').click()
accept_button = driver.find_element(By.XPATH,'//*[@id="productForm-pricing"]/div/div[3]/div[2]/div[1]/div/div[2]/div/div[2]/label/input')
accept_button_classess = accept_button.get_attribute("class")
print(accept_button_classess)

'''
df = pd.DataFrame()
if "ng-touched" in accept_button_classess:
    print(df.loc[df['Product ID (DO NOT EDIT)'] == sku_id])
    accept_button.click()

driver.find_element(By.XPATH,'//*[@id="product_form"]/div[1]/div[1]/div/span[2]/button/span[2]').click()
print("Saved changes, completed")'''

In [ ]:
def click_procedure_missing_pre_order(driver, sku_id):
    print("Now browsing to SKU: " + sku_id)
    driver.get("https://admin.shoplineapp.com/admin/waddystore/products/"+sku_id+"/edit")

    driver.find_element(By.XPATH,'//*[@id="product_form"]/div[1]/div[3]/ul/li[4]/a').click()
    print("Go to Price and Quantity Tab")

    accept_button = driver.find_element(By.XPATH,'//*[@id="productForm-pricing"]/div/div[3]/div[2]/div[1]/div/div[2]/div/div[2]/label/input')
    accept_button_classess = accept_button.get_attribute("class")
    print(accept_button_classess)
    if "ng-touched" in accept_button_classess:
        
        driver.find_element(By.XPATH,'//*[@id="product_form"]/div[1]/div[3]/ul/li[8]/a').click()
        print("Go to Settings Tab")
        pre_order_switch = driver.find_element(By.XPATH,'//*[@id="productForm-settings"]/div[1]/div[3]/div[1]/div/div[2]/div/div[1]/div')
        pre_order_switch_classess = pre_order_switch.get_attribute("class")
        if "switch-off" in pre_order_switch_classess:
            driver.find_element(By.XPATH,'//*[@id="productForm-settings"]/div[1]/div[3]/div[1]/div/div[2]/div/div[1]/div/span[2]').click()
            print("Switched on Preorder Product Setting")
            
            pre_order_msg_english = "This product is a pre-order product. It will arrive in about 7-14 working days, thank you for your patient! (AVAILABLE does not mean in stock)"
            pre_order_msg_chinese = "此商品為預購商品，大約7-14工作天到貨，請耐心等候♡（尚有庫存不代表有現貨）"
            
            english_msg_box = driver.find_element(By.XPATH,'//*[@id="productForm-settings"]/div[1]/div[3]/div[2]/div/div[2]/div/input')
            english_msg_box.send_keys(Keys.CONTROL, 'a')
            english_msg_box.send_keys(pre_order_msg_english)
            print("Typed in Preorder Product Note (English)")
            chinese_msg_box = driver.find_element(By.XPATH,'//*[@id="productForm-settings"]/div[1]/div[3]/div[3]/div/div[2]/div/input')
            chinese_msg_box.send_keys(Keys.CONTROL, 'a')
            chinese_msg_box.send_keys(pre_order_msg_chinese)
            print("Typed in Preorder Product Note (Chinese)")

            driver.find_element(By.XPATH,'//*[@id="product_form"]/div[1]/div[1]/div/span[2]/button/span[2]').click()
            print("Saved changes, completed")
        return "Pre order settings"
    else:
        return "No Missing Pre order settings"

# cross check with stock level
os.chdir("C:/Users/prolo/OneDrive/Documents/Code_Project/Clicker/codespaces-flask/Check")
file_path = os.listdir()[0]
stock_df = pd.read_excel(file_path,index_col=None, header=0)


os.chdir("C:/Users/prolo/OneDrive/Documents/Code_Project/Clicker/codespaces-flask/Update")
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith(".xls"):
        file_path = f"{file}" 
        df = pd.DataFrame()
        df = pd.read_excel(file_path,index_col=None, header=0)
        df = df.drop(index=0)
        ofs_df, preorder_df = pd.DataFrame(), pd.DataFrame()
        preorder_df = df[df['Preorder Product'] == "N"]
        stock_level = preorder_df['Quantity (DO NOT EDIT)']
        ofs_df = preorder_df[preorder_df['Quantity (DO NOT EDIT)'] <= 0]
        ofs_df = preorder_df[stock_level <= 0]
        for barcode in ofs_df["Barcode"]:
                check_stock_df = stock_df.loc[stock_df['商品條碼'] == barcode]
                if int(check_stock_df['預設倉庫']) > 0:
                    tmp_df = pd.DataFrame()
                    tmp_df = df.loc[df['Barcode'] == barcode]
                    if tmp_df.shape[0] > 0:
                        for sku_id in tmp_df['Product ID (DO NOT EDIT)']:
                            msg = click_procedure_missing_pre_order(driver, sku_id)
                            print(msg)

